In [1]:
from __future__ import print_function, division

In [2]:
from numpy import *

In [3]:
import scipy.spatial

In [4]:
## Generate a random affine matrix
def gen_random_affine():
    return random.random( (3,4) )

## Generate an affine identity matrix
def gen_identity_affine():
    return identity(4)[:3]

In [5]:
## Let's create some poses. The first (0-th) pose will be the identity.
num_handles = 4
num_poses = 3

poses_handles = []

poses_handles.append( [ gen_identity_affine() for i in range(num_handles) ] )
for pose in range( 1, num_poses ):
    poses_handles.append( [ gen_random_affine() for i in range(num_handles) ] )

## The dimensions of poses_handles are: #handles by #poses by affine 3x4 matrix
poses_handles = asarray( poses_handles )
poses_handles = swapaxes( poses_handles, 0, 1 )
print( poses_handles.shape )

(4, 3, 3, 4)


In [6]:
## Let's create some per-vertex skinning weights. Each vertex has a weight for each handle.
num_vertices = 100
## Start with random weights
weights = random.random( ( num_vertices, num_handles ) )
## Make the weights sum to 1. For each successive weight, scale it by whatever is left.
for h in range( 1, num_handles-1 ):
    weights[:,h] *= (1 - weights[:,:h].sum(axis=1))
## Make the last weight be whatever is left.
weights[:,-1] = (1 - weights[:,:-1].sum(axis=1))

## Overwrite the last num_handles vertices with binary vectors elements
ensure_corners = True
if ensure_corners:
    for h in range( num_handles ):
        weights[-(h+1)] = 0.
        weights[-(h+1),h] = 1.
    
    print( weights[-num_handles:] )

print( "Maximum weight for each handle:", weights.max( axis = 0 ) )

## The dimensions of weights are #vertices by #handles.

## The weights all be positive.
assert weights.min() >= 0

## The weights must sum to one. Otherwise, we should normalize them once.
assert abs( weights.sum( axis = 1 ) - 1. < 1e-7 ).all()

[[ 0.  0.  0.  1.]
 [ 0.  0.  1.  0.]
 [ 0.  1.  0.  0.]
 [ 1.  0.  0.  0.]]
Maximum weight for each handle: [ 1.  1.  1.  1.]


In [7]:
## The lbs() function generates the weighted average transformation.

## For all poses at once.
def lbs_all_poses( poses_handles, weights ):
    ## The weights across handles must sum to one.
    assert abs( weights.sum( axis = 1 ) - 1. < 1e-7 ).all()
    
    return ( weights[:,:,newaxis,newaxis,newaxis] * poses_handles[newaxis,...] ).sum( axis = 1 )

## For one set of handles (e.g. one pose) at a time.
def lbs_one_pose( handles, weights ):
    return lbs_all_poses( handles[:,newaxis,...], weights )

In [8]:
## lbs_one_pose() on pose 0 should give us an identity matrix for every vertex.
lbs_one_pose( poses_handles[:,0], weights )

array([[[[ 1.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.],
         [ 0.,  0.,  1.,  0.]]],


       [[[ 1.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.],
         [ 0.,  0.,  1.,  0.]]],


       [[[ 1.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.],
         [ 0.,  0.,  1.,  0.]]],


       ..., 
       [[[ 1.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.],
         [ 0.,  0.,  1.,  0.]]],


       [[[ 1.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.],
         [ 0.,  0.,  1.,  0.]]],


       [[[ 1.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.],
         [ 0.,  0.,  1.,  0.]]]])

In [9]:
poses = lbs_all_poses( poses_handles, weights )

In [10]:
## Doesn't work because it's in a linear subspace
# pose = lbs_one_pose( poses_handles[:,2], weights )
# hull = scipy.spatial.ConvexHull( pose.reshape( num_vertices, -1 ) )

In [11]:
## Doesn't work because it's in a linear subspace
# hull = scipy.spatial.ConvexHull( poses.reshape( num_vertices, -1 ) )

In [12]:
## What are the dimensions of our poses? It should be #verts by #poses by 3x4 affine matrix.
print( poses.shape )
## Flattened it should be #vertices by (#poses*3*4)
X = poses.reshape( num_vertices, num_poses * 3 * 4 )
print( X.shape )

(100, 3, 3, 4)
(100, 36)


In [13]:
## Use SVD to get the lower dimensional space.
## The number of non-zero singular values is the number of handles for perfect reconstruction.
U, s, V = linalg.svd( X-average(X,axis=0)[newaxis,:], full_matrices = False, compute_uv = True )

In [14]:
## What are the singular values?
s.round(4)

array([ 4.2802,  3.072 ,  2.3298,  0.    ,  0.    ,  0.    ,  0.    ,
        0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
        0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
        0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
        0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ])

In [15]:
## Find the first index less than a threshold:
len(s)-searchsorted( s[::-1], 1e-6 )

3

In [16]:
## SVD reconstructs the original data
abs( X - (( U @ diag(s) @ V )+average(X,axis=0)[newaxis,:]) ).max()

6.106226635438361e-16

In [17]:
## Formalize the above with functions.
def uncorrellated_pose_space( poses, threshold = 1e-6 ):
    X = poses.reshape( num_vertices, num_poses * 3 * 4 )
    
    ## Subtract the average.
    Xavg = average( X, axis = 0 )[newaxis,:]
    Xp = X - Xavg
    
    U, s, V = linalg.svd( Xp, full_matrices = False, compute_uv = True )
    
    ## The first index less than threshold
    stop_s = len(s) - searchsorted( s[::-1], threshold )
    
    def restore( uncorrellated_poses ):
        return ( ( uncorrellated_poses @ V[:stop_s] ) + Xavg ).reshape( -1, num_poses, 3, 4 )
    
    return Xp @ V[:stop_s].T, restore

In [18]:
## Vertify that we can go back and forth with these functions
uncorrelated_poses, restore = uncorrellated_pose_space( poses )
print( uncorrelated_poses.shape )
abs( restore( uncorrelated_poses ) - poses ).max()

(100, 3)


6.6613381477509392e-16

In [19]:
uncorrelated_poses

array([[-0.39638115,  0.03788546, -0.1059153 ],
       [ 0.19340268, -0.13301294,  0.04758523],
       [ 0.61666752, -0.06286895,  0.88127594],
       [-0.11641443,  0.1992854 , -0.03367275],
       [-0.22364434,  0.09463676, -0.13508763],
       [ 0.82127827,  0.16550535, -0.05823267],
       [ 0.05742205, -0.41785488,  0.21688607],
       [ 0.5498841 ,  0.05540272, -0.07345759],
       [ 0.12114186,  0.12486178, -0.14240731],
       [-0.01793153,  0.02889626, -0.1592491 ],
       [ 0.34600728,  0.19088603,  0.05623336],
       [ 0.172957  , -0.24029927,  0.18790088],
       [-0.30368347,  0.09085669, -0.01155345],
       [-0.21536088,  0.03047973,  0.36811391],
       [ 0.22330151,  0.25350264, -0.17305519],
       [-0.27330656, -0.98274899, -0.17795676],
       [-0.46647074,  0.00689704, -0.02195907],
       [-0.46046944,  0.23346185, -0.07741276],
       [ 0.3215589 ,  0.08646158,  0.18639685],
       [ 0.69180122,  0.0631264 , -0.08324361],
       [-0.36770756,  0.1762138 ,  0.147

In [20]:
## Now let's use the convex hull on the uncorrelated poses.
hull = scipy.spatial.ConvexHull( uncorrelated_poses )
print( len( hull.vertices ) )
print( hull.vertices )

4
[96 97 98 99]


In [21]:
poses_handles_restored = restore( uncorrelated_poses[ hull.vertices ] )
print( poses_handles_restored.shape )
poses_handles_restored.round(4)

(4, 3, 3, 4)


array([[[[ 1.    ,  0.    ,  0.    , -0.    ],
         [ 0.    ,  1.    ,  0.    ,  0.    ],
         [ 0.    ,  0.    ,  1.    ,  0.    ]],

        [[ 0.1618,  0.6628,  0.0279,  0.7518],
         [ 0.4068,  0.5128,  0.7024,  0.0608],
         [ 0.9883,  0.3409,  0.5049,  0.003 ]],

        [[ 0.5488,  0.5334,  0.6492,  0.7641],
         [ 0.5698,  0.1277,  0.9963,  0.72  ],
         [ 0.3122,  0.4389,  0.0477,  0.7558]]],


       [[[ 1.    ,  0.    ,  0.    ,  0.    ],
         [ 0.    ,  1.    ,  0.    ,  0.    ],
         [ 0.    ,  0.    ,  1.    ,  0.    ]],

        [[ 0.9466,  0.0022,  0.9126,  0.3161],
         [ 0.8186,  0.2353,  0.5777,  0.3695],
         [ 0.8214,  0.5324,  0.3489,  0.7171]],

        [[ 0.5099,  0.1697,  0.8957,  0.3783],
         [ 0.151 ,  0.6171,  0.1714,  0.858 ],
         [ 0.6432,  0.9361,  0.9321,  0.9387]]],


       [[[ 1.    ,  0.    ,  0.    ,  0.    ],
         [ 0.    ,  1.    ,  0.    ,  0.    ],
         [ 0.    ,  0.    ,  1.    ,  0.    

In [22]:
print( poses_handles.shape )
poses_handles

(4, 3, 3, 4)


array([[[[ 1.        ,  0.        ,  0.        ,  0.        ],
         [ 0.        ,  1.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  1.        ,  0.        ]],

        [[ 0.56603496,  0.0338491 ,  0.25409268,  0.27796005],
         [ 0.70453787,  0.03534301,  0.22477817,  0.02495022],
         [ 0.24143388,  0.78449893,  0.12492566,  0.38591782]],

        [[ 0.04387591,  0.20914108,  0.5740765 ,  0.82752929],
         [ 0.35078339,  0.28681667,  0.56446772,  0.94757487],
         [ 0.77064615,  0.37257277,  0.35870931,  0.3025911 ]]],


       [[[ 1.        ,  0.        ,  0.        ,  0.        ],
         [ 0.        ,  1.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  1.        ,  0.        ]],

        [[ 0.82810258,  0.17313714,  0.93067704,  0.71426722],
         [ 0.80752166,  0.5420642 ,  0.11125642,  0.04806   ],
         [ 0.14056286,  0.33436254,  0.18624416,  0.54033129]],

        [[ 0.78117757,  0.20053239,  0.8795

In [23]:
## Find the closest handle in the original data, pose_handles, to the restored data.
for i, handles in enumerate( poses_handles ):
    diffs = ( abs( poses_handles_restored - handles[newaxis,...] ).reshape( poses_handles_restored.shape[0], -1 ).sum( axis = 1 ) )
    print( "Original data index", i, "best matches restored data index", diffs.argmin(), "with error", diffs.min() )

Original data index 0 best matches restored data index 3 with error 3.71808157845e-15
Original data index 1 best matches restored data index 2 with error 4.55818398924e-15
Original data index 2 best matches restored data index 1 with error 4.05897861518e-15
Original data index 3 best matches restored data index 0 with error 3.93755438216e-15


In [24]:
## Find the closest handle in the restored data to the original pose_handles.
for i, handles in enumerate( poses_handles_restored ):
    diffs = ( abs( poses_handles - handles[newaxis,...] ).reshape( poses_handles.shape[0], -1 ).sum( axis = 1 ) )
    print( "Restored data index", i, "best matches original data index", diffs.argmin(), "with error", diffs.min() )

Restored data index 0 best matches original data index 3 with error 3.93755438216e-15
Restored data index 1 best matches original data index 2 with error 4.05897861518e-15
Restored data index 2 best matches original data index 1 with error 4.55818398924e-15
Restored data index 3 best matches original data index 0 with error 3.71808157845e-15


In [25]:
## Plotting in 3D. This only works because PCA found our data was in 3D.
%matplotlib tk
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(*uncorrelated_poses.T)
plt.show()